# **Линейная регрессия: численное решение**

✍ Ранее мы с вами установили, что аналитическое решение линейной регрессии — метод наименьших квадратов — имеет несколько недостатков. Основной из них — большая вычислительная сложность обращения матриц.

Хотелось бы получить метод, который справляется с этим недостатком. Для этого давайте вернёмся на шаг назад и вспомним, как вообще звучит постановка задачи поиска параметров модели.

В самом простом двумерном случае мы пытаемся найти такие коэффициенты  уравнения прямой w0 и w1, чтобы средний квадрат ошибки (MSE) был минимален.

![](data\f38.png)

Гауссу удалось найти общее решение для этой оптимизационной задачи и вывести формулу метода наименьших квадратов для поиска коэффициентов.

*Но можно пойти другим путём, не аналитическим (вывод формулы «в лоб»), а численным (итерационное приближение функции к минимуму).*

Самым популярным численным методом оптимизации, используемым в машинном обучении, является **алгоритм градиентного спуска**.

**Градиентный спуск (Gradient descent)** — самый используемый *алгоритм* минимизации функции потерь. Он применяется почти в каждой модели машинного обучения и является наиболее простым в реализации из всех методов численной оптимизации.
***
Начнём обзор алгоритма немного издалека.

Представим, что мы находимся на некоторой холмистой местности и нам надо добраться до самой низкой точки, но делаем мы это вслепую, то есть не знаем сам ландшафт. Назовём эту точку целью.

![](https://lms.skillfactory.ru/assets/courseware/v1/955db3eb396297eabf11b61019fb49d5/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_1.png)

Давайте опишем, как мы будем искать эту точку. Для начала необходимо задать начальную точку, из которой мы, собственно, стартуем. Далее мы двигаемся в сторону крутизны склона. Если склон круче справа, надо сделать шаг вправо, если склон круче слева, надо сделать шаг влево. Повторяем шаги до тех пор, пока не достигнем самой низкой точки.

У вас мог возникнуть вопрос: как определить, что мы достигли цели — самой низкой точки? В этой точке крутизна склона с обеих сторон равна 0 или близка к нему (ровная поверхность). Можно использовать эту информацию как точку остановки нашего алгоритма.
***
Описанный нами алгоритм можно перевести на язык математики. Он то и будет называться **алгоритмом градиентного спуска**.

Наша функция потерь, которая зависит от параметров модели, — это аналогия ландшафта местности. Пространство, в котором находится ландшафт, — это пространство параметров нашей модели. То есть это система координат, в которой по осям отложены все возможные значения параметров.

В двумерном случае, когда есть только один параметр, от которого зависит функция потерь, можно построить график функции потерь. Например, для MSE, зависящей от одного параметра, график будет иметь вид **параболы**:

![](https://lms.skillfactory.ru/assets/courseware/v1/2031eae68e1cfcfbfa6642928a844ef8/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_2.png)

Если параметров не один, а два, то функция потерь будет графически представлена в виде **поверхности в трёхмерном пространстве**. Ниже приведён пример такой поверхности и её вид сверху в виде концентрических кругов:

![](https://lms.skillfactory.ru/assets/courseware/v1/86c47a39251c3d192a314187ec8d9043/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_3.png)

В общем случае, когда у нас больше параметров модели, мы будем работать в многомерном пространстве. Но в этом нет ничего страшного. Суть поиска минимума от этого не меняется, меняется только сложность функции — **структуры ландшафта**.

Для линейной регрессии необходимо найти в этом пространстве такие координаты w0, w1, w2, ... ,wm в которых находится минимум функции потерь.

Как же нам понять, в какую сторону двигаться? Что будет отвечать за направление крутизны склона? На этот вопрос нам ответит математический анализ. В теории анализа функций, зависящих от нескольких переменных, существует понятие градиента.

***Математически градиент — это вектор, который состоит из частных производных от всех параметров функции (те которые w0,w1,w2 ... , w0).***

Он записывается следующим образом:

![](data\f39.png)

где  ***L(w)*** — функция потерь, зависящая от параметров модели, функция может быть любой (например, MSE). ***∇*** — символ набла — символьное сокращение градиента, читается как ***«градиент функции L(w)»***. 

Пусть пока что математическая формализация градиента нам непонятна, но нам важно отметить его ключевую особенность.

                    Градиент — это вектор, который показывает направление наискорейшего роста функции, а его длина — это само значение скорости функции в точке.

Если вновь обратиться к примеру с холмами, градиент показывает, с какой скоростью и в каком направлении нужно двигаться из текущей точки, чтобы достичь более высшей точки.
***
А теперь время фокусов. Если поставить перед градиентом знак минус ***—∇L(w)***, то мы получим вектор **антиградиента**, который показывает в сторону наискорейшего убывания функции потерь! А это и есть то, что нам нужно. С помощью этого знания мы сможем вычислять следующую координату в пространстве — следующую точку, которую нам нужно посетить, чтобы дойти до цели — минимума функции.

В случае одного параметра:

![](https://lms.skillfactory.ru/assets/courseware/v1/c28fb31ed69592e3b38db4d47492aeb3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_4.png)

В случае двух параметров:

![](https://lms.skillfactory.ru/assets/courseware/v1/4a12d561ec2f0938cfd2838ffc9dce62/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_5.png)

Формально это записывается следующим образом (формулу запоминать не нужно):

![](data\f40.png)

где w — это вектор параметров модели, координаты в пространстве, а индекс в круглых скобках сверху означает номер точки в пространстве. Запись ***∇L(w^(k))*** означает, что градиент вычисляется в текущей точке под номером k.

Согласно приведённой формуле, новая координата ***w^(k+1)*** в пространстве параметров определяется как текущая координата ***w^(k)*** минус **скорость роста** в текущей точке ***∇L(w^(k))***, помноженная на **коэффициент «скольжения»**.

Отдельное внимание стоит уделить коэффициенту ***η*** (читается как «эта»). Это поправочный коэффициент, который носит название **темп обучения (learning rate)**.

**Темп обучения** — это основной параметр алгоритма. Он **определяет то, насколько сильно мы будем двигать точку**. В аналогии с нашим примером с движением по холмам можно сказать, что это коэффициент, обратный сопротивлению ландшафта, по которому мы движемся.

Управляя данным параметром (уменьшая и увеличивая его), мы управляем скоростью движения к точке минимума. Чем больше темп обучения, тем длиннее наши шаги и тем быстрее мы движемся, и наоборот. О том, зачем этот параметр нужен и как выбирать значение ***η***, мы поговорим чуть позже.
***
**Примечание. Темп обучения является примером внешнего параметра алгоритма, которым может управлять пользователь. Такие параметры ещё называют гиперпараметрами.**
***

Есть ещё одно важное свойство градиента: теоретически в точке минимума длина вектора равна 0, то есть движения не происходит. Это свойство мы можем использовать в качестве критерия остановки нашего алгоритма.

![](https://lms.skillfactory.ru/assets/courseware/v1/c06a98caf800ce3fb0f82a6a26235f05/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_6.png)